# ABD final project: Data handling
## Python version
3.5 onwards

## Modules needed
* pandas
* numpy
* openpyxl

In [1]:
# Uncoment to install modules inside the notebook
# !pip install pandas
# !pip install openpyxl

import openpyxl
import numpy as np
import pandas as pd

from pandas import Series, DataFrame

# DataFrame formatting

## Nginx default log format
```
'$remote_addr - $remote_user [$time_local] "$request" '
'$status $body_bytes_sent "$http_referer" '
'"$http_user_agent" "$http_x_forwarded_for"'
```                      

## Formating DataFrames
After loading the data and the output DataFrame

### DataFrame after data load
Reading with a space character as the separation:

row | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10
--- | --- | --- | --- | --- |--- |--- |--- |--- |--- |--- |---
\# | \$remote_addr | - | \$remote_user | [\$time_local(time) | \$time_local(locale)] | \$request | \$status | \$body_bytes_sent | \$http_referer | \$http_user_agent | \$http_x_forwarded_for

### Output DataFrame
After the formatting, ready to the human read and fill:

row | http_user_agent | request_method | status | request_url | is_evil
--- | --- | --- | --- | --- |--- 
\# | String | GET or POST | int | String | Empty String


In [2]:
# Loading the full log history in a DataFrame
df_data = pd.read_csv('data/full_log.log', sep=" ", header=None)

In [3]:
# Joining $time_local(time) and $time_local(locale) columns in a single 'time' column
df_data['time'] = df_data[3] + ' ' + df_data[4]

# Splitting $request column in three: 'method', 'url' and 'protocol' columns
df_data['method'], df_data['url'], df_data['protocol'] = df_data[5].str.split(' ', 2).str

# Dropping '-', $remote_user, $time_local(time), $time_local(locale), $request and $body_bytes_sent columns
df_data.drop([1, 2, 3, 4, 5, 7], axis=1, inplace=True)

In [4]:
# Showing DataFrame length
print('rows before deleting:', len(df_data))

# Declaring an auxiliar DataFrame to get the rows with at least 1 NaN value
df_aux = df_data[df_data.isna().any(axis=1)]

# Dropping the rows with null values
df_data.dropna(inplace=True)

# Showing deleted and final lengths
print('rows deleted:', len(df_aux))
print('final rows:', len(df_data))

rows before deleting: 7823
rows deleted: 60
final rows: 7763


In [5]:
# Declaring an auxiliar DataFrame clonning the main DataFrame
df_aux = df_data.copy()
# Replace the '-' character with NaN in the $http_referer column in the auxiliar DataFrame
df_aux[8].replace('-', pd.np.nan, inplace=True)
# Getting all the rows with no NaN value in the $http_referer column in the auxiliar DataFrame
df_aux = df_aux[~df_aux.isna().any(axis=1)]

# Printing the auxiliar DataFrame length
print('there was', len(df_aux), 'http_referer notnull records')

# Dropping the $http_referer column
df_data.drop([8], axis=1, inplace=True)

there was 110 http_referer notnull records


In [6]:
# Declaring an auxiliar DataFrame clonning the main DataFrame
df_aux = df_data.copy()
# Replace the '-' character with NaN in the $http_x_forwarded_for column in the auxiliar DataFrame
df_aux[10].replace('-', pd.np.nan, inplace=True)
# Getting all the rows with no NaN value in the $http_x_forwarded_for column in the auxiliar DataFrame
df_aux = df_aux[~df_aux.isna().any(axis=1)]

# Printing the auxiliar DataFrame length
print('there was', len(df_aux), 'http_x_forwarded_for notnull records')

# Dropping the $http_x_forwarded_for column
df_data.drop([10], axis=1, inplace=True)

there was 13 http_x_forwarded_for notnull records


In [7]:
# Declaring an auxiliar integer with the length of the DataFrame as value
int_aux = len(df_data)

# Replace the 'GET' or 'POST' Strings with NaN in the 'method' column in the DataFrame
df_data['method'].replace(r'^((?!GET|POST).)*$', pd.np.nan, regex=True, inplace=True)

# Dropping the rows with null values
df_data.dropna(inplace=True)

# Printing the deleted rows count
print('there was', int_aux - len(df_data), 'non POST or GET records')

there was 46 non POST or GET records


In [8]:
# Renaiming the columns
df_data.columns = ['remote_addr', 'status', 'http_user_agent', 'time_local', 'request_method', 'request_url', 'request_protocol']
#df_aux.columns = ['remote_addr', 'status', 'http_user_agent', 'time_local', 'request_method', 'request_url', 'request_protocol']

# Dropping the 'remote_addr', 'time_local', 'request_protocol' columns
df_data.drop(['remote_addr', 'time_local', 'request_protocol'], axis=1, inplace=True)

In [9]:
# Replacing the '-' character with '<NO_DATA>' in the 'http_user_agent' column to be more legible
df_data['http_user_agent'].replace('-', '<NO_DATA>', inplace=True)

# Reordering the columns
df_data = df_data[['http_user_agent', 'request_method', 'status', 'request_url']]

In [10]:
# Resetting the DataFrame indexes
df_data = df_data.reset_index(drop=True)

# Creating a new row for the human log evaluating
df_data['is_evil'] = ''

In [11]:
# Filling some 'is_evil' values based on common words present on attacks to OUR server

# Declaring search counters
cont = 0
cont2 = 0
cont3 = 0
cont4 = 0
# Declaring total counter
cont_t = 0
# Declaring search Strings
q_str = 'php'
q_str2 = 'busybox'
q_str3 = 'xml'
q_str4 = 'robots.txt'

# Iterating the DataFrame
for i in df_data.itertuples():
    is_counted = False
    if q_str in i[4]:
        cont += 1
        df_data.at[i[0], 'is_evil'] = '1'
        if not is_counted:
            cont_t += 1
            is_counted = True
    if q_str2 in i[4]:
        cont2 += 1
        df_data.at[i[0], 'is_evil'] = '1'
        if not is_counted:
            cont_t += 1
            is_counted = True
    if q_str3 in i[4]:
        cont3 += 1
        df_data.at[i[0], 'is_evil'] = '1'
        if not is_counted:
            cont_t += 1
            is_counted = True
    if q_str4 in i[4]:
        cont4 += 1
        df_data.at[i[0], 'is_evil'] = '1'
        if not is_counted:
            cont_t += 1
            is_counted = True

# Printing the results
print('there are', cont, '\'' + q_str + '\'', 'urls,', cont2, '\'' + q_str2 + '\'', 'urls,', cont3, '\'' + q_str3 + '\'', 'urls and', cont4, '\'' + q_str4 + '\'', 'urls')
print('total evilized records are', cont_t, 'from', len(df_data))
print('leaving', len(df_data) - cont_t, 'records to edit')

there are 6757 'php' urls, 4 'busybox' urls, 1457 'xml' urls and 51 'robots.txt' urls
total evilized records are 6866 from 7717
leaving 851 records to edit


In [12]:
# Sorting by 'is_evil' column
df_data.sort_values('is_evil', inplace=True)

# Exporting the DataFrame to an Excel file
df_data.to_excel('data/log_data.xlsx')

# Printing the number of rows
print('total rows:',len(df_data))

total rows: 7717
